In [1]:
import numpy as np
import pandas as pd
import pypsa
import highspy 

%matplotlib inline

In [2]:
DATA_FILE = 'pypsa-eu-uk-2030.xlsx'

In [3]:
load = pd.read_excel(DATA_FILE, sheet_name='load', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
pv = pd.read_excel(DATA_FILE, sheet_name='pv', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
wind = pd.read_excel(DATA_FILE, sheet_name='wind', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
wind_offshore = pd.read_excel(DATA_FILE, sheet_name='wind_offshore', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
ror = pd.read_excel(DATA_FILE, sheet_name='ror', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
inflow = pd.read_excel(DATA_FILE, sheet_name='inflow', parse_dates=['t'], index_col='t',usecols='A:AK').dropna(axis=1, how='all').dropna()
chp = pd.read_excel(DATA_FILE, sheet_name='chp', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()
chp_bio = pd.read_excel(DATA_FILE, sheet_name='chp_bio', parse_dates=['t'], index_col='t').dropna(axis=1, how='all').dropna()

buses = pd.read_excel(DATA_FILE, sheet_name='buses', index_col=0, usecols='A:G')
cbf_buses = pd.read_excel(DATA_FILE, sheet_name='cbf_buses', index_col=0, usecols='A:F')
lines = pd.read_excel(DATA_FILE, sheet_name='lines', index_col='name').dropna(axis=1, how='all').dropna()
links = pd.read_excel(DATA_FILE, sheet_name='links', index_col=0, usecols='A:G').dropna(axis=1, how='all').dropna()
gen_pv = pd.read_excel(DATA_FILE, sheet_name='gen_pv', index_col=0, usecols='A:E')
gen_wind = pd.read_excel(DATA_FILE, sheet_name='gen_wind', index_col=0, usecols='A:E')
gen_wind_offshore = pd.read_excel(DATA_FILE, sheet_name='gen_wind_offshore', index_col=0, usecols='A:E')
gen_gas = pd.read_excel(DATA_FILE, sheet_name='gen_gas', index_col=0, usecols='A:J')
gen_oil = pd.read_excel(DATA_FILE, sheet_name='gen_oil', index_col=0, usecols='A:J')
gen_coal = pd.read_excel(DATA_FILE, sheet_name='gen_coal', index_col=0,usecols='A:J')
gen_nuclear = pd.read_excel(DATA_FILE, sheet_name='gen_nuclear', index_col=0, usecols='A:I')
gen_biomass = pd.read_excel(DATA_FILE, sheet_name='gen_biomass', index_col=0, usecols='A:G')
gen_biogas = pd.read_excel(DATA_FILE, sheet_name='gen_biogas', index_col=0, usecols='A:G')
gen_ror = pd.read_excel(DATA_FILE, sheet_name='gen_ror', index_col=0, usecols='A:E')
gen_reservoir = pd.read_excel(DATA_FILE, sheet_name='gen_reservoir', index_col=0, usecols='A:E')
gen_other_res = pd.read_excel(DATA_FILE, sheet_name='gen_other_res', index_col=0, usecols='A:F')
gen_dsr = pd.read_excel(DATA_FILE, sheet_name='gen_dsr', index_col=0, usecols='A:F')
gen_gas_chp = pd.read_excel(DATA_FILE, sheet_name='gen_gas_chp', index_col=0, usecols='A:J')
gen_coal_chp = pd.read_excel(DATA_FILE, sheet_name='gen_coal_chp', index_col=0, usecols='A:J')
gen_oil_chp = pd.read_excel(DATA_FILE, sheet_name='gen_oil_chp', index_col=0, usecols='A:G')
gen_res_chp = pd.read_excel(DATA_FILE, sheet_name='gen_res_chp', index_col=0, usecols='A:G')
gen_bio_chp = pd.read_excel(DATA_FILE, sheet_name='gen_bio_chp', index_col=0, usecols='A:G')
gen_cbf = pd.read_excel(DATA_FILE, sheet_name='gen_cbf', index_col=0, usecols='A:E')
st_hps = pd.read_excel(DATA_FILE, sheet_name='st_hps', index_col=0, usecols='A:I')
st_battery = pd.read_excel(DATA_FILE, sheet_name='st_battery', index_col=0, usecols='A:I')
st_other = pd.read_excel(DATA_FILE, sheet_name='st_other', index_col=0, usecols='A:I')

In [4]:
gas_chp_timeseries=pd.DataFrame(chp,columns=gen_gas_chp.index)
coal_chp_timeseries=pd.DataFrame(chp,columns=gen_coal_chp.index)
oil_chp_timeseries=pd.DataFrame(chp,columns=gen_oil_chp.index)
res_chp_timeseries=pd.DataFrame(chp,columns=gen_res_chp.index)

for col in gas_chp_timeseries.columns:
    gas_chp_timeseries[col].values[:] = chp['chp']

for col in coal_chp_timeseries.columns:
    coal_chp_timeseries[col].values[:] = chp['chp']

for col in oil_chp_timeseries.columns:
    oil_chp_timeseries[col].values[:] = chp['chp']

for col in res_chp_timeseries.columns:
    res_chp_timeseries[col].values[:] = chp['chp']
  

In [5]:
network = pypsa.Network()
network.set_snapshots(load.index)

network.import_components_from_dataframe(buses, 'Bus')
network.import_components_from_dataframe(cbf_buses, 'Bus')

network.import_components_from_dataframe(lines, 'Line')
network.lines['s_nom_extendable'] = True  

network.madd("Link", links.index,
             bus0=links['bus0'].tolist(), bus1= links['bus1'].tolist(),
             p_nom=links['p_nom'].tolist())

network.madd("Generator", gen_cbf.index, suffix='_CBF', carrier='CBF',
             bus=gen_cbf.index.tolist(), p_nom=gen_cbf['p_nom'].to_list(), p_nom_extendable=False, 
             marginal_cost=gen_cbf['marginal_cost'].to_list())

dict_carriers = {
    'Lignite': 0.334, 'Hard coal': 0.354, 'Gas': 0.187, 'Gas CCS': 0, 'Oil': 0.334, 'Hydrogen': 0, 'Biomass': 0.403, 'Biogas': 0.178, 'BECCS': 0,
    'Geothermal': 0.026, 'Wind': 0, 'Wind offshore': 0, 'PV': 0, 'HPS': 0, 'Hydro': 0, 'Other RES': 0, 'CBF': 0, 'VOLL': 0, 'Battery': 0, 'Other storage' : 0, 'Nuclear': 0
}

for carrier in dict_carriers:
    network.add("Carrier", name=carrier, co2_emissions=dict_carriers[carrier])

network.madd("Load", load.columns, bus=load.columns, p_set=load)

Index(['EN_NorthEast', 'EN_NorthWest', 'EN_Yorkshire', 'EN_EastMidlands',
       'EN_WestMidlands', 'EN_East', 'EN_London', 'EN_SouthEast',
       'EN_SouthWest', 'EN_Wales', 'Scotland', 'NorthernIreland', 'Austria',
       'Belgium', 'Bulgaria', 'Croatia', 'Czechia', 'Denmark', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy',
       'Latvia', 'Lithuania', 'Netherlands', 'Poland', 'Portugal', 'Romania',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden'],
      dtype='object')

In [6]:
network.madd('Generator', 
      gen_pv['bus'],
      suffix='_PV',
      bus=gen_pv['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_pv['p_nom'].to_list(),
      carrier='PV', 
      marginal_cost=gen_pv['marginal_cost'].to_list(), 
      p_max_pu=pv)

network.madd('Generator', 
      gen_wind['bus'],
      suffix='_Wind',
      bus=gen_wind['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_wind['p_nom'].to_list(), 
      carrier='Wind', 
      marginal_cost=gen_wind['marginal_cost'].to_list(), 
      p_max_pu=wind)

network.madd('Generator', 
      gen_wind_offshore['bus'],
      suffix='_Wind_offshore',
      bus=gen_wind_offshore['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_wind_offshore['p_nom'].to_list(), 
      carrier='Wind offshore', 
      marginal_cost=gen_wind_offshore['marginal_cost'].to_list(), 
      p_max_pu=wind_offshore)

network.madd('Generator', 
      gen_gas.index,
      bus=gen_gas['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_gas['p_nom'].to_list(), 
      carrier=gen_gas['carrier'].to_list(), 
      marginal_cost=gen_gas['marginal_cost'].to_list(),
      efficiency=gen_gas['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_oil.index,
      bus=gen_oil['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_oil['p_nom'].to_list(), 
      carrier='Oil', 
      marginal_cost=gen_oil['marginal_cost'].to_list(),
      efficiency=gen_oil['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_coal.index,
      bus=gen_coal['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_coal['p_nom'].to_list(), 
      carrier=gen_coal['carrier'].to_list(), 
      marginal_cost=gen_coal['marginal_cost'].to_list(),
      efficiency=gen_coal['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_nuclear.index,
      bus=gen_nuclear['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_nuclear['p_nom'].to_list(), 
      carrier='Nuclear', 
      marginal_cost=gen_nuclear['marginal_cost'].to_list(),
      efficiency=gen_nuclear['efficiency'].to_list(),
      p_max_pu=gen_nuclear['p_max_pu'].to_list(),
      p_min_pu=gen_nuclear['p_min_pu'].to_list()
      )

network.madd('Generator', 
      gen_biomass.index,
      bus=gen_biomass['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_biomass['p_nom'].to_list(), 
      carrier=gen_biomass['carrier'].to_list(), 
      marginal_cost=gen_biomass['marginal_cost'].to_list(),
      efficiency=gen_biomass['efficiency'].to_list(),
      p_max_pu=gen_biomass['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_biogas['bus'],
      suffix='_Biogas',
      bus=gen_biogas['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_biogas['p_nom'].to_list(), 
      carrier='Biogas', 
      marginal_cost=gen_biogas['marginal_cost'].to_list(),
      efficiency=gen_biogas['efficiency'].to_list(),
      p_max_pu=gen_biogas['p_max_pu'].to_list()
      )

network.madd('Generator', 
       gen_ror.index,
       bus=gen_ror['bus'].to_list(), 
       p_nom_extendable=False, 
       p_nom=gen_ror['p_nom'].to_list(), 
       carrier=gen_ror['carrier'].to_list(),
       marginal_cost=gen_ror['marginal_cost'].to_list(),
       p_max_pu=ror
       )

network.madd('Generator', 
       gen_reservoir.index,
       bus=gen_reservoir['bus'].to_list(), 
       p_nom_extendable=False, 
       p_nom=gen_reservoir['p_nom'].to_list(), 
       carrier=gen_reservoir['carrier'].to_list(),
       marginal_cost=gen_reservoir['marginal_cost'].to_list(),
       p_max_pu=inflow
       )

network.madd('Generator', 
      gen_other_res['bus'],
      suffix='_OtherRES',
      bus=gen_other_res['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_other_res['p_nom'].to_list(), 
      carrier='Other RES', 
      marginal_cost=gen_other_res['marginal_cost'].to_list(),
      p_max_pu=gen_other_res['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_dsr['bus'],
      suffix='_DSR',
      bus=gen_dsr['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=50000,#gen_dsr['p_nom'].to_list(), 
      carrier='DSR', 
      marginal_cost=gen_dsr['marginal_cost'].to_list(),
      p_max_pu=gen_dsr['p_max_pu'].to_list()
      )

network.madd('Generator', 
      gen_gas_chp.index,
      bus=gen_gas_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_gas_chp['p_nom'].to_list(), 
      carrier=gen_gas_chp['carrier'].to_list(), 
      marginal_cost=gen_gas_chp['marginal_cost'].to_list(),
      p_max_pu=1.32*gas_chp_timeseries,
      p_min_pu=gas_chp_timeseries,
      efficiency=gen_gas_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_coal_chp.index,
      bus=gen_coal_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_coal_chp['p_nom'].to_list(), 
      carrier=gen_coal_chp['carrier'].to_list(), 
      marginal_cost=gen_coal_chp['marginal_cost'].to_list(),
      p_max_pu=coal_chp_timeseries,
      p_min_pu=0.9*coal_chp_timeseries,
      efficiency=gen_coal_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_oil_chp.index,
      bus=gen_oil_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_oil_chp['p_nom'].to_list(), 
      carrier=gen_oil_chp['carrier'].to_list(), 
      marginal_cost=gen_oil_chp['marginal_cost'].to_list(),
      p_max_pu=1.32*oil_chp_timeseries,
      p_min_pu=oil_chp_timeseries,
      efficiency=gen_oil_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_res_chp.index,
      bus=gen_res_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_res_chp['p_nom'].to_list(), 
      carrier=gen_res_chp['carrier'].to_list(), 
      marginal_cost=gen_res_chp['marginal_cost'].to_list(),
      p_max_pu=1.32*res_chp_timeseries,
      p_min_pu=res_chp_timeseries,
      efficiency=gen_res_chp['efficiency'].to_list()
      )

network.madd('Generator', 
      gen_bio_chp.index,
      bus=gen_bio_chp['bus'].to_list(), 
      p_nom_extendable=False, 
      p_nom=gen_bio_chp['p_nom'].to_list(), 
      carrier=gen_bio_chp['carrier'].to_list(), 
      marginal_cost=gen_bio_chp['marginal_cost'].to_list(),
      p_max_pu=chp_bio,
      p_min_pu=chp_bio*0.9,
      efficiency=gen_bio_chp['efficiency'].to_list()
      )

network.madd("StorageUnit", st_hps.index, bus=st_hps['bus'].tolist(), carrier=st_hps['carrier'].tolist(),
             p_nom=st_hps['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_hps['max_hours'].to_list(),
             p_max_pu=st_hps['p_max_pu'].tolist(),
             efficiency_dispatch=st_hps['efficiency_dispatch'].tolist(),
             standing_loss=st_hps['standing_loss'].tolist())
            
network.madd("StorageUnit", st_battery.index, bus=st_battery['bus'].tolist(), carrier=st_battery['carrier'].tolist(),
             p_nom=st_battery['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_battery['max_hours'].to_list(),
             p_max_pu=st_battery['p_max_pu'].tolist(),
             efficiency_dispatch=st_battery['efficiency_dispatch'].tolist(),
             standing_loss=st_battery['standing_loss'].tolist())

network.madd("StorageUnit", st_other.index, bus=st_other['bus'].tolist(), carrier=st_other['carrier'].tolist(),
             p_nom=st_other['p_nom'].tolist(), p_nom_extendable=False, max_hours=st_other['max_hours'].to_list(),
             p_max_pu=st_other['p_max_pu'].tolist(),
             efficiency_dispatch=st_other['efficiency_dispatch'].tolist(),
             standing_loss=st_other['standing_loss'].tolist())


Index(['EN_NorthEast_OtherStorage', 'EN_NorthWest_OtherStorage',
       'EN_Yorkshire_OtherStorage', 'EN_EastMidlands_OtherStorage',
       'EN_WestMidlands_OtherStorage', 'EN_East_OtherStorage',
       'EN_London_OtherStorage', 'EN_SouthEast_OtherStorage',
       'EN_SouthWest_OtherStorage', 'EN_Wales_OtherStorage',
       'Scotland_OtherStorage', 'NorthernIreland_OtherStorage',
       'Bulgaria_Other storage', 'Croatia_Other storage',
       'Czechia_Other storage', 'Estonia_Other storage',
       'Hungary_Other storage', 'Latvia_Other storage',
       'Lithuania_Other storage', 'Poland_Other storage',
       'Romania_Other storage', 'Slovakia_Other storage',
       'Slovenia_Other storage'],
      dtype='object', name='name')

In [7]:
network.optimize(solver_name='highs')

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
INFO:linopy.model: Solve linear problem using Highs solver
INFO:linopy.io:Writing objective.
Writing variables.: 100%|████████████████████████████████████████████████████████████████| 6/6 [00:26<00:00,  4.45s/it]
INFO:linopy.io: Writing time: 185.24s
INFO:linopy.solvers:Log file at C:\Users\harri\AppData\Local\Temp\highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 19368360 primals, 39858000 duals
Objective: 1.32e+11
Solver model: available
Solver message: optimal



('ok', 'optimal')

In [96]:
replace_regions_UK = {'EN_SouthEast':'UK', 'EN_East':'UK','EN_EastMidlands':'UK','EN_London':'UK',
              'EN_NorthEast':'UK', 'EN_NorthWest':'UK', 'EN_SouthEast':'UK','EN_SouthWest':'UK', 'EN_Wales':'UK',
              'EN_WestMidlands':'UK', 'EN_Yorkshire':'UK', 'NorthernIreland':'UK', 'Scotland':'UK'}


In [8]:
p_by_carrier = network.generators_t.p.groupby(network.generators.carrier, axis=1).sum()
generation_by_fuel_yearly = p_by_carrier.sum()/1000000
generation_by_fuel_yearly

carrier
BECCS              8.672296
Biogas            78.750965
Biomass          127.307318
CBF                0.243637
DSR               66.295173
Gas              286.831996
Gas CCS            1.101931
Hard coal        226.699162
Hydro            222.679381
Hydrogen          19.712915
Lignite          153.727840
Nuclear          525.262040
Oil              192.672739
Other RES        124.455740
PV               672.532272
Wind             710.733041
Wind offshore    389.906498
dtype: float64

In [ ]:
#network.buses_t.marginal_price.to_csv('prices_uk_eu.csv')

In [74]:
p_by_carrier = network.generators_t.p.groupby([network.generators.carrier, network.generators.bus], axis=1).sum()
generation_by_fuel_by_country = p_by_carrier.sum()/1000000
generation_by_fuel_by_country = generation_by_fuel_by_country.reset_index()
generation_by_fuel_by_country['bus'] = generation_by_fuel_by_country['bus'].replace(replace_regions_UK)
generation_by_fuel_by_country.to_csv('generation_by_fuel_by_country.csv', index=False)

In [10]:
p_by_unit = network.generators_t.p.sum()/1000000
generation_by_unit = p_by_unit
generation_by_unit.to_csv('generation_by_unit.csv')

In [139]:
def create_to_from_link(df_row):
    from_to_list = df_row['Link'].split('_')
    country_from = from_to_list[1]
    country_to = from_to_list[2]
    return country_from, country_to

In [142]:
generation_by_link = network.links_t.p0.sum()/1000000
generation_by_link = generation_by_link.reset_index()
generation_by_link['Link'] = generation_by_link['Link'].str.replace('*EN_', 'UK', regex=True)
#generation_by_link[['country_from', 'country_to']] = generation_by_link.apply(create_to_from_link, axis=1, result_type='expand')
#generation_by_link.to_csv('generation_by_link.csv')

error: nothing to repeat at position 0

In [158]:
generation_by_link['Link']

0      link_FR_EN_SouthEast
1      link_NL_EN_SouthEast
2          link_IE_EN_Wales
3      link_BE_EN_SouthEast
4      link_NO_EN_NorthEast
               ...         
111              link_GR_TR
112              link_RU_EE
113              link_RU_FI
114              link_RU_LT
115              link_RU_LV
Name: Link, Length: 116, dtype: object

In [141]:
generation_by_link

,Link,0,country_from,country_to
0,link_FR_EN_SouthEast,27.533796,FR,EN
1,link_NL_EN_SouthEast,5.828994,NL,EN
2,link_IE_EN_Wales,9.113425,IE,EN
3,link_BE_EN_SouthEast,0.181918,BE,EN
4,link_NO_EN_NorthEast,10.626631,NO,EN
...,...,...,...,...
111,link_GR_TR,1.784896,GR,TR
112,link_RU_EE,0.000000,RU,EE
113,link_RU_FI,0.000000,RU,FI
114,link_RU_LT,0.000000,RU,LT


In [ ]:
domestic_generation_by_fuel_yearly = generation_by_fuel_yearly
domestic_generation_by_fuel_yearly=domestic_generation_by_fuel_yearly.drop('CBF')
generation_by_fuel_yearly_percentage = domestic_generation_by_fuel_yearly.div(domestic_generation_by_fuel_yearly.sum(), axis=0)
generation_by_fuel_yearly_percentage = generation_by_fuel_yearly_percentage.round(4)
generation_by_fuel_yearly_percentage = (generation_by_fuel_yearly_percentage * 100).astype(str) + '%'
generation_by_fuel_yearly_percentage


In [12]:
capacity_by_fuel = network.generators.groupby("carrier")["p_nom"].sum()
capacity_by_fuel

carrier
BECCS            1.100000e+03
Biogas           1.374849e+04
Biomass          2.087151e+04
CBF              2.000000e+05
DSR              1.800000e+06
Gas              1.887161e+05
Gas CCS          2.630000e+03
Hard coal        3.516500e+04
Hydro            1.105667e+05
Hydrogen         6.486750e+03
Lignite          2.374300e+04
Nuclear          9.816400e+04
Oil              2.732600e+04
Other RES        1.881438e+04
PV               6.618083e+05
Wind             3.935331e+05
Wind offshore    1.636507e+05
Name: p_nom, dtype: float64

In [13]:
storage_capacity = network.storage_units.groupby("carrier")["p_nom"].sum()
storage_capacity

carrier
Battery          110480.544078
HPS               53367.000000
Hydro              4888.000000
Other storage     27512.562879
Name: p_nom, dtype: float64

In [14]:
generation_carriers = pd.merge(network.generators, network.carriers, left_on="carrier", right_index=True)
emissions = network.generators_t.p * generation_carriers["co2_emissions"] / generation_carriers["efficiency"]

(emissions.sum().sum()*1000000000000/1000000)/(generation_by_fuel_yearly.sum()*1000000000)

231.7111237274765

In [15]:
emissions.sum().sum()/1000000

882.2597864377361